# Clustering - 군집 분석

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import mglearn

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 K-Means

[K-Means](https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-7-K-%ED%8F%89%EA%B7%A0-%EA%B5%B0%EC%A7%91%ED%99%94-K-means-Clustering)

- 거리 기반 군집 알고리즘

In [ ]:
# 입력 데이터와 k-평균 군집 알고리즘이 세번 진행되기까지의 과정
mglearn.plots.plot_kmeans_algorithm()

In [ ]:
# k-평균 알고리즘으로 찾은 클러스터 중심과 클러스터 경계
mglearn.plots.plot_kmeans_boundaries()

### 1.1 K-Means 알고리즘

#### 1.1.1 데이터 생성

In [ ]:
from sklearn.datasets import make_blobs

# 2차원 데이터 생성
X, y = make_blobs(random_state=1)

#### 1.1.2 K-Means 군집 모델 생성

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

In [ ]:
print(kmeans.labels_)

In [ ]:
# 군집 예측
print(kmeans.predict(X))

In [ ]:
# 각 군집 중심과의 거리
print(kmeans.transform(X))

#### 1.1.3 K-Means 군집 모델 - 클러스터 중심과 클러스터 할당

In [ ]:
mglearn.discrete_scatter(X[:,0], X[:,1], kmeans.labels_, markers='o')

mglearn.discrete_scatter(
    kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], [0, 1, 2],
    markers='^', markeredgewidth=2)
    
plt.title('k-평균 알고리즘으로 찾은 세개의 클러스터 중심과 클러스터 할당')
plt.show()

#### 1.1.4 클러스터 수 조정 - 2, 5

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# 두 개의 클러스터 중심을 사용합니다
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
assignments = kmeans.labels_

mglearn.discrete_scatter(X[:,0], X[:,1], assignments, ax=axes[0])

# 다섯 개의 클러스터 중심을 사용합니다
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)
assignments = kmeans.labels_

mglearn.discrete_scatter(X[:,0], X[:,1], assignments, ax=axes[1])
plt.show()

### 1.2 k-평균 알고리즘이 실패하는 경우

#### 1.2.1 밀도가 다른 데이터

In [ ]:
# 데이터 생성
from sklearn.datasets import make_blobs

X_varied, y_varied = make_blobs(n_samples=200,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=170)

In [ ]:
# K-Means 알고리즘 적용
y_pred = KMeans(n_clusters=3, random_state=0).fit_predict(X_varied)

mglearn.discrete_scatter(X_varied[:,0], X_varied[:,1], y_pred)

plt.title('클러스터 밀도가 다를 때 k-평균으로 찾은 클러스터 할당')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.legend(['클러스터 0', '클러스터 1', '클러스터 2'], loc='best')
plt.show()

#### 1.2.2 원형이 아닌 데이터

In [ ]:
# 무작위로 클러스터 데이터 생성
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

# 데이터가 길게 늘어지도록 변경
transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)

In [ ]:
# 세 개의 클러스터로 데이터에 KMeans 알고리즘 적용
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
y_pred = kmeans.predict(X)

In [ ]:
# 클러스터 할당과 클러스터 중심을 나타냅니다
mglearn.discrete_scatter(X[:,0], X[:,1], kmeans.labels_, markers='o')
mglearn.discrete_scatter(
    kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], [0, 1, 2],
    markers='^', markeredgewidth=2)
    
plt.title('원형이 아닌 클러스터를 구분하지 못하는 k-평균 알고리즘')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

#### 1.2.3 복잡한 모양의 데이터

In [ ]:
# two_moons 데이터를 생성(노이즈 추가)
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

In [ ]:
# 두 개의 클러스터로 데이터에 K-Means 알고리즘 적용
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
y_pred = kmeans.predict(X)

In [ ]:
# 클러스터 할당과 클러스터 중심을 표시합니다
plt.scatter(X[:,0], X[:,1], c=y_pred, cmap=mglearn.cm2, s=60, edgecolors='k')
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1],
            marker='^', c=[mglearn.cm2(0), mglearn.cm2(1)], s=100, linewidth=2, edgecolors='k')
            
plt.title('복잡한 모양의 클러스터를 구분하지 못하는 k-평균 알고리즘')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

# 2 병합군집(Agglomerative Clustering)

[병합군집](https://velog.io/@khsfun0312/Hierarchical-Clustering)

#### linkage 옵션
- ward: 기본값인 ward 연결은 모든 클러스터 내의 분산을 가장 작게 증가시키는 두 클러스터를 병합. 크기가 비교적 비슷한 클러스터 생성
- average: 클러스터 포인트 사이의 평균 거리가 가장 짧은 두 클러스터를 병합
- complete: 클러스터 포인트 사이의 최대 거리가 가장 짧은 두 클러스터를 병합

In [ ]:
# 병합 군집 진행 과정
mglearn.plots.plot_agglomerative_algorithm()

### 2.1 병합군집(Agglomerative Clustering)

In [ ]:
# 데이터 생성
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=1)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X)

mglearn.discrete_scatter(X[:,0], X[:,1], assignment)

plt.title('병합 군집을 사용한 세개의 클러스터 할당')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.legend(['클러스터 0', '클러스터 1', '클러스터 2'], loc='best')
plt.show()

### 2.2 계층적 군집과 덴드로그램(dendrograms)

In [ ]:
# 병합 군집으로 생성한 계층적 군집
mglearn.plots.plot_agglomerative()

In [ ]:
# 데이터 생성
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=0, n_samples=12)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, ward

# 덴드로 그램
linkage_array = ward(X)
dendrogram(linkage_array)

# 두 개와 세 개의 클러스터를 구분하는 커트라인을 표시
ax = plt.gca()
bounds = ax.get_xbound()
ax.plot(bounds, [7.25, 7.25], '--', c='k')
ax.plot(bounds, [4, 4], '--', c='k')

ax.text(bounds[1], 7.25, ' 두 개 클러스터', va='center', fontdict={'size': 15})
ax.text(bounds[1], 4, ' 세 개 클러스터', va='center', fontdict={'size': 15})

plt.xlabel('샘플 번호')
plt.ylabel('클러스터 거리')
plt.show()

# 3 GMM(Gaussian Mixture Model)

[GMM](https://untitledtblog.tistory.com/133)

- 확률 기반 군집 알고리즘

In [ ]:
# 무작위로 클러스터 데이터 생성
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

# 데이터가 길게 늘어지도록 변경
transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)

In [ ]:
# 세개의 클러스터로 GMM 알고리즘 적용
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3, random_state=0)
gmm_label = gmm.fit(X).predict(X)

In [ ]:
# 클러스터 할당 시각화
mglearn.discrete_scatter(X[:,0], X[:,1], gmm_label, markers='o')

plt.title('원형이 아닌 클러스터에 GMM 적용')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

# 4 DBSCAN

[DBSCAN](https://bcho.tistory.com/1205)

#### 옵션
- 입실론 주변 영역(epsilon): 개별 데이터를 중심으로 입실론 반경을 가지는 원형의 영역
- 최소 데이터 개수(min points): 개별 데이터의 입실론 주변 영역에 포함되는 타 데이터의 개수

#### 포인트 구분
- 핵심 포인트(Core Point): 주변 영역 내에 최소 데이터 개수 이상의 타 데이터를 가지고 있는 데이터 포인트
- 이웃 포인트(Neighbor Point): 주변 영역 내에 위치한 타 데이터 포인트
- 경계 포인트(Border Point): 핵심 포인트를 이웃 포인트로 가지고 있는 데이터 포인트
- 잡음 포인트(Noise Point): 최소 데이터 개수 이상의 이웃 포인트를 가지고 있지 않고 핵심 포인트를 이웃 포인트로도 가지고 있지 않은 데이터 포인트

### 4.1 DBSCAN 적용

In [ ]:
# 데이터 생성
from sklearn.datasets import make_blobs
X, y = make_blobs(random_state=0, n_samples=12)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
clusters = dbscan.fit_predict(X)

print('클러스터 레이블:\n', clusters)

In [ ]:
# min_samples, eps 옵션에 따른 포인트 표시
mglearn.plots.plot_dbscan()

### 4.2 DBSCAN 적용 - 데이터 스케일링

In [ ]:
# 데이터 생성
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

In [ ]:
# 스케일 조정
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
clusters = dbscan.fit_predict(X_scaled)

In [ ]:
# 클러스터 할당 시각화

plt.scatter(X_scaled[:,0], X_scaled[:,1], c=clusters, cmap=mglearn.cm2, s=60, edgecolors='black')

plt.title('기본값 eps=0.5를 사용해 DBSCAN으로 찾은 클러스터 할당')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

### 4.3 DBSCAN 적용 - 원형 데이터

In [ ]:
# 데이터 생성
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=1000, shuffle=True, noise=0.05, random_state=0, factor=0.5)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.2, min_samples=10, metric='euclidean')
clusters = dbscan.fit_predict(X)

In [ ]:
# 클러스터 할당 시각화

plt.scatter(X[:,0], X[:,1], c=clusters, cmap=mglearn.cm2, s=60, edgecolors='black')

plt.title('원형 데이터 분포를 DBSCAN으로 찾은 클러스터 할당')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

# 5 군집 알고리즘 평가

### 5.1 타겟값이 있는 경우
- ARI(Adjusted Rand Index)
- NMI(Normalized Mutual Information)

In [ ]:
# 데이터 생성
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

In [ ]:
# 스케일 조정
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

from sklearn.metrics.cluster import adjusted_rand_score

In [ ]:
# 군집 알고리즘 별 평가 - ARI

fig, axes = plt.subplots(1, 4, figsize=(15, 3), subplot_kw={'xticks': (), 'yticks': ()})

# 비교를 위해 무작위로 클러스터 할당을 합니다
random_state = np.random.RandomState(seed=0)
random_clusters = random_state.randint(low=0, high=2, size=len(X))

# 무작위 할당한 클러스터를 그립니다
axes[0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=random_clusters, cmap=mglearn.cm3, s=60, edgecolors='black')
axes[0].set_title('무작위 할당 - ARI: {:.2f}'.format(adjusted_rand_score(y, random_clusters)))

# 사용할 알고리즘 모델을 리스트로 만듭니다
algorithms = [KMeans(n_clusters=2), AgglomerativeClustering(n_clusters=2), DBSCAN()]

for ax, algorithm in zip(axes[1:], algorithms):
    # 클러스터 할당과 클러스터 중심을 그립니다
    clusters = algorithm.fit_predict(X_scaled)
    ax.scatter(X_scaled[:,0], X_scaled[:,1], c=clusters, cmap=mglearn.cm3, s=60, edgecolors='black')
    ax.set_title('{} - ARI: {:.2f}'.format(algorithm.__class__.__name__, adjusted_rand_score(y, clusters)))

#### 5.1.1 정확도 - ARI 비교

In [ ]:
from sklearn.metrics import accuracy_score

# 포인트가 클러스터로 나뉜 두 가지 경우
clusters1 = [0, 0, 1, 1, 0]
clusters2 = [1, 1, 0, 0, 1]

# 모든 레이블이 달라졌으므로 정확도는 0입니다
print('정확도: {:.2f}'.format(accuracy_score(clusters1, clusters2)))

# 같은 포인트가 클러스터에 모였으므로 ARI는 1입니다
print('ARI: {:.2f}'.format(adjusted_rand_score(clusters1, clusters2)))

### 5.2 타겟값이 없는 경우
- 실루엣 계수(Silhouette Coefficient)

In [ ]:
# 데이터 생성
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

In [ ]:
# 스케일 조정
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
from sklearn.metrics.cluster import silhouette_score

In [ ]:
# 군집 알고리즘 별 평가 - Silhouette Coefficient

fig, axes = plt.subplots(1, 4, figsize=(15, 3), subplot_kw={'xticks': (), 'yticks': ()})

# 비교를 위해 무작위로 클러스터 할당을 합니다
random_state = np.random.RandomState(seed=0)
random_clusters = random_state.randint(low=0, high=2, size=len(X))

# 무작위 할당한 클러스터를 그립니다
axes[0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=random_clusters, cmap=mglearn.cm3, s=60, edgecolors='black')
axes[0].set_title('무작위 할당: {:.2f}'.format(silhouette_score(X_scaled, random_clusters)))

# 사용할 알고리즘 모델을 리스트로 만듭니다
algorithms = [KMeans(n_clusters=2), AgglomerativeClustering(n_clusters=2), DBSCAN()]

for ax, algorithm in zip(axes[1:], algorithms):
    clusters = algorithm.fit_predict(X_scaled)
    # 클러스터 할당과 클러스터 중심을 그립니다
    ax.scatter(X_scaled[:, 0], X_scaled[:, 1], c=clusters, cmap=mglearn.cm3, s=60, edgecolors='black')
    ax.set_title('{} : {:.2f}'.format(algorithm.__class__.__name__, silhouette_score(X_scaled, clusters)))

---

In [ ]:
# End of file